In [1]:
import numba
import numpy as np

In [2]:
def mod_cumsum(xs, p):
    ret = [0]
    for x in xs[:-1]:
        ret.append((ret[-1] + x) % p)
    return ret

def count_seqs_in_columns(p, n):
    idx = 0
    col = [1]

    ub = 1
    while ub <= n:
        ub *= p
    ub *= p * p
    
    seqs = set()
    col_cnts = []
    for _ in range(ub):
        extended_col = []
        while len(col) - 1 + n > len(extended_col):
            extended_col.extend(col)
        extended_col = tuple(extended_col)
        
        col_cnt = 0
        for i in range(len(col)):
            cur_seq = extended_col[i : i + n]
            if cur_seq not in seqs:
                seqs.add(cur_seq)
                col_cnt += 1

        col_cnts.append(col_cnt)

        nxt_col = mod_cumsum(col, p)
        nxt_col[0] = 0
        if (col[-1] + nxt_col[-1]) % p != 0:
            pcol = []
            for _ in range(p):
                pcol.extend(col)
            nxt_col = mod_cumsum(pcol, p)

        assert (col[-1] + nxt_col[-1]) % p == 0

        col = nxt_col
        
    while col_cnts[-1] == 0:
        del col_cnts[-1]
    
    return col_cnts

count_seqs_in_columns(p=3, n=2)

[1, 3, 2, 1, 2]

In [3]:
%%time
p = 2
old = []
for n in range(1, 100):
    cnts = count_seqs_in_columns(p=2, n=n)
    while len(old) < len(cnts):
        old.append(0)
    
    diffs = [a - b for a, b in zip(cnts, old)]
    diffs[n] -= 1
    str_diffs = "".join([str(x)[-1] if x > 0 else " " for x in diffs])
    print(f"{n:3} {str_diffs[n:]}")
    
    old = cnts

  1  
  2  
  3  
  4 2
  5 2
  6 2 2
  7 222
  8 422
  9 422
 10 422   2
 11 422  22
 12 422 222
 13 4222222
 14 4242222
 15 4442222
 16 6442222
 17 6442222
 18 6442222       2
 19 6442222      22
 20 6442222     222
 21 6442222    2222
 22 6442222   22222
 23 6442222  222222
 24 6442222 2222222
 25 644222222222222
 26 644222422222222
 27 644224422222222
 28 644244422222222
 29 644444422222222
 30 646444422222222
 31 666444422222222
 32 866444422222222
 33 866444422222222
 34 866444422222222               2
 35 866444422222222              22
 36 866444422222222             222
 37 866444422222222            2222
 38 866444422222222           22222
 39 866444422222222          222222
 40 866444422222222         2222222
 41 866444422222222        22222222
 42 866444422222222       222222222
 43 866444422222222      2222222222
 44 866444422222222     22222222222
 45 866444422222222    222222222222
 46 866444422222222   2222222222222
 47 866444422222222  22222222222222
 48 86644442222222

In [8]:
%%time
old = []
for n in range(1, 64):
    cnts = count_seqs_in_columns(p=3, n=n)
    while len(old) < len(cnts):
        old.append(0)
    
    diffs = [a - b for a, b in zip(cnts, old)]
    str_diffs = "".join([str(x)[-1] if x > 0 else " " for x in diffs])
    print(f"{n:3} {str_diffs[n:]}")
    
    old = cnts

  1 2
  2 212
  3 6121   1 31
  4 4111 111321
  5 41114111321  1           1     3  1
  6 41154111321 11          11    33 11
  7 41554111321111         111   333111
  8 43552111321111        1111  2332111
  9 62532111311111     1 11111 32322111
 10 52332111111111   111111111432222111
 11 52332111111111  4111111111432222111        1                                   1                 3        1
 12 52332111111111 44111111111432222111       11                                  11                33       11
 13 52332111111111444111111111432222111      111                                 111               333      111
 14 52332111111115444111111111432222111     1111                                1111              3333     1111
 15 52332111111155444111111111432222111    11111                               11111             33333    11111
 16 52332111111555444111111111432222111   111111                              111111            333333   111111
 17 52332111115555444111111111432222111  1